# Natural Language Processing with RNNs and Attention

In this notebook we will explore the character RNN, stateless RNN, stateful RNN, RNN for sentimental analysis, Encoder-Decoder architecture and finally Transformer.

## Setup

In [31]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


## Character RNN

### Splitting a sequence into batches of shuffled windows
For example, let's split the sequence 0 to 14 into windows of length 5, each shifted by 2 (e.g.,[0, 1, 2, 3, 4], [2, 3, 4, 5, 6], etc.), then shuffle them, and split them into inputs (the first 4 steps) and targets (the last 4 steps) (e.g., [2, 3, 4, 5, 6] would be split into [[2, 3, 4, 5], [3, 4, 5, 6]]), then create batches of 3 such input/target pairs:

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda windows: (windows[:-1], windows[1:]))
dataset = dataset.batch(3).prefetch(1)

for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


In [33]:
shakespeare_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read() 

In [34]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [35]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [36]:
# char_level => character level encoding. default is word level encoding
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) 
tokenizer.fit_on_texts([shakespeare_text])

In [37]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [38]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [39]:
max_id = len(tokenizer.word_index) # no. of distinct characters
dataset_size = tokenizer.document_count # total number of characters

Let's encode the full text so each character is represented by its ID (we subtract 1 to get IDs from 0 to 38, rather than form 1 to 39).

In [40]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [41]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [42]:
dataset = dataset.prefetch(1)

In [43]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

## Training the Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

history = model.fit(dataset, epochs=20)

### Using the Model
We first need to preprocess it:

In [44]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprecess(['How are yo'])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

### Generating fake Shakespearean text
Until now, we would predict the next 'letter' in the text. To generate new text using the Char-RNN model, we could feed it some text, make the model predict the most likely next letter, add it at the end of the text, then give the extended text to the model to guess the next letter, and so on. But in practice this often leads tot he same words being repeated over and over again.

Instead we can pick the next character randomly. 

In [45]:
# X_new = preprocess([text])
# ... to be continued

## Stateful RNN
In stateless RNN, at each training iteration the model starts with a hidden state full of zeroes, then it updates this state at each time step, and after the last time step, it throws it away, as it is not needed anymore.

We can preserve the final state after processing one training batch and use it as the initial state for the next training batch. This way model can learn long-term patterns despite only backpropagating through short sequences. This is called *stateful RNN*.

**Note**: stateful RNN only makes sense if each input sequence in a batch starts exactly where the corresponding sequence in the previous batch left off. Hence we will have to use sequential nonoverlapping input sequences.

In [55]:
tf.random.set_seed(42)

In [61]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [62]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)


In [63]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

At the end of each epoch, we need to reset the states before we go back to the beginning of the text. For this we use a callback:

In [64]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

## Sentimental Analysis
We will use IMDB reviews dataset.

In [68]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 2s 0us/step


In [76]:
X_train[0][:10] # the dataset is already preprocessed.

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [81]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token
    
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [82]:
import tensorflow_datasets as tfds

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)

Shuffling and writing examples to /Users/sankalp/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB5T8M1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/sankalp/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB5T8M1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/sankalp/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB5T8M1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/sankalp/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [86]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [95]:
train_size = info.splits['train'].num_examples
test_size = info.splits["test"].num_examples

train_size, test_size

(25000, 25000)

In [96]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [97]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ") # replacing <br /> tags   
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z]", b" ") # replacing any characters other than a-z
    X_batch = tf.strings.split(X_batch) # split by spaces
    # converting ragged tensor to dense tensor and padding for same length
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch 

In [115]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(8, 60, 1), dtype=string, numpy=
 array([[[b'Red'],
         [b'Eye'],
         [b'is'],
         [b'not'],
         [b'the'],
         [b'kind'],
         [b'of'],
         [b'movie'],
         [b'that'],
         [b's'],
         [b'going'],
         [b'to'],
         [b'win'],
         [b'the'],
         [b'Palme'],
         [b'D'],
         [b'or'],
         [b'but'],
         [b'Wes'],
         [b'Craven'],
         [b'has'],
         [b'never'],
         [b'been'],
         [b'that'],
         [b'kind'],
         [b'of'],
         [b'director'],
         [b'anyway'],
         [b'and'],
         [b'his'],
         [b'branding'],
         [b'is'],
         [b'a'],
         [b'good'],
         [b'indication'],
         [b'of'],
         [b'what'],
         [b'a'],
         [b'film'],
         [b'goer'],
         [b'can'],
         [b'expect'],
         [b'The'],
         [b'fact'],
         [b'that'],
         [b'Red'],
         [b'Eye'],
         [b'is'],
       

We need to construct a vocabulary: go through the whole training set once, applying preprocess() function, and using a `Counter` to count the no. of occurrences of each word.

In [98]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [99]:
vocabulary.most_common()[:3]

[(b'<pad>', 224494), (b'the', 61156), (b'a', 38569)]

Keeping only the 10000 common words:

In [100]:
vocab_size = 10000

truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]
]

Now, adding preprocessing step to replace each word and replacing with its ID, we will create a lookup table using 1000 oov buckets:

In [110]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)

num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [111]:
table.lookup(tf.constant([b'This movie was faaaaaaantastic'.split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   24,    12,    13, 10770]])>

In [112]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [113]:
train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [114]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])

In [117]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 122s 156ms/step - loss: 0.6713 - accuracy: 0.5507
Epoch 2/5
782/782 [==============================] - 124s 158ms/step - loss: 0.4195 - accuracy: 0.8056
Epoch 3/5
782/782 [==============================] - 118s 151ms/step - loss: 0.2785 - accuracy: 0.8876
Epoch 4/5
782/782 [==============================] - 107s 137ms/step - loss: 0.1882 - accuracy: 0.9311
Epoch 5/5
782/782 [==============================] - 5422s 7s/step - loss: 0.1496 - accuracy: 0.9468


#### Masking
In order to ignore the padding tokens add mask_zero=True when creating Embedding layer.

### Reusing Pretrained Embeddings
The Tensorflow Hub project makes it easy to reuse pretrained model components in your own models. These model components are called *modules*. Browse the [TF Hub repository](https://tfhub.dev).

In [121]:
tf.random.set_seed(42)

In [123]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [125]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1',
                  dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/saved_model.pb
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.data-00000-of-00001
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.index
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/assets/tokens.txt


Next, we can just load the IMDB review dataset and directly train the model:

In [128]:
datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
batch_size = 32
train_set = datasets['train'].batch(batch_size).prefetch(1)

history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)

Epoch 1/5
781/781 [==============================] - 120s 154ms/step - loss: 0.5460 - accuracy: 0.7267
Epoch 2/5
  1/781 [..............................] - ETA: 0s - loss: 0.7954 - accuracy: 0.7500WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 3905 batches). You may need to use the repeat() function when building your dataset.


  1/781 [..............................] - 0s 304ms/step - loss: 0.7954 - accuracy: 0.7500


## An Encoder-Decoder Network for Neural Machine Translation
Translating English sentences to French:

##### how it works?
The English sentences are fed to the encoder, and the decoder outputs the French transalations. Note that the French translations are also used as inputs to the decoder, but shifted back by one step. In other words, the decoder is given as input the word that it *should* have output at the previous step.

Note that the English sentence will be reversed before they are fed to the encoder.

In [129]:
tf.random.set_seed(42)

In [130]:
vocab_size = 100
embed_size = 10

Tensorflow Addons project includes many sequence-to-sequence tools to let you easily build production-ready Encoder-Decoders.

In [132]:
import tensorflow_addons as tfa

In [136]:
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])


In [137]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [138]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Epoch 1/2
32/32 [==============================] - 7s 221ms/step - loss: 4.6052
Epoch 2/2
32/32 [==============================] - 7s 212ms/step - loss: 4.6031
